# Day 2: Building Agents - Hands-On Notebook

Welcome to the Day 2 hands-on notebook! Today, we will dive into the OpenAI Agents SDK to build and orchestrate agents.

## Activity 1: Your First SDK Agent & Conversational State

First, ensure you have the `openai-agents` library installed (`pip install openai-agents`).

In [ ]:
!pip install openai-agents -q

In [ ]:
from agents import Agent, Runner
import asyncio

# Define a simple agent
math_tutor_agent = Agent(
    name="Math Tutor",
    instructions="You are a friendly math tutor. You help students understand math concepts by explaining them clearly and providing examples."
)

async def run_agent():
    # Run the agent with an initial prompt
    result = await Runner.run(math_tutor_agent, "Can you explain the Pythagorean theorem?")
    print(f"[Tutor]: {result.final_output}")

    # Now, let's handle conversational state by feeding the history back in
    conversation_history = result.to_input_list()
    conversation_history.append({"role": "user", "content": "Can you give me an example?"})

    follow_up_result = await Runner.run(math_tutor_agent, conversation_history)
    print(f"[Tutor]: {follow_up_result.final_output}")

# In a Jupyter Notebook, you can run the async function like this:
await run_agent()

## Activity 2: Empowering Agents with Tools

Let's give our agent a tool to perform a specific action. We'll create a simple calculator tool.

In [ ]:
from agents import function_tool

@function_tool
def simple_calculator(expression: str) -> str:
    ""Calculates the result of a simple mathematical expression (e.g., '5*3').""
    try:
        # WARNING: eval is not safe for production use with untrusted input!
        # This is for demonstration purposes only.
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"

# Create an agent with the tool
calculator_agent = Agent(
    name="Calculator Agent",
    instructions="You are a helpful assistant with a calculator. When asked to compute something, use your calculator tool.",
    tools=[simple_calculator]
)

async def run_calculator_agent():
    result = await Runner.run(calculator_agent, "What is 125 * 8?")
    print(result.final_output)

await run_calculator_agent()

## Activity 3: Designing a Multi-Agent System

Now, let's build a system where a triage agent hands off tasks to specialized agents.

In [ ]:
# Specialist Agents
shipping_agent = Agent(
    name="Shipping Specialist",
    instructions="You are an expert on package tracking and shipping logistics. You can provide tracking updates and delivery estimates."
)

product_agent = Agent(
    name="Product Specialist",
    instructions="You have deep knowledge of the company's product catalog. You can answer questions about product features, specifications, and availability."
)

# Triage Agent with Handoffs
triage_agent = Agent(
    name="Triage Agent",
    instructions="You are the first point of contact. Understand the user's needs and hand them off to the correct specialist: either the Shipping Specialist or the Product Specialist.",
    handoffs=[shipping_agent, product_agent]
)

async def run_triage():
    # Test a shipping query
    shipping_query = "Where is my package with tracking number 12345XYZ?"
    result1 = await Runner.run(triage_agent, shipping_query)
    print(f"Query: '{shipping_query}'")
    print(f"Handed off to: {result1.last_agent.name}")
    print(f"Final Response: {result1.final_output}
")

    # Test a product query
    product_query = "Does the new X-100 model have bluetooth?"
    result2 = await Runner.run(triage_agent, product_query)
    print(f"Query: '{product_query}'")
    print(f"Handed off to: {result2.last_agent.name}")
    print(f"Final Response: {result2.final_output}")

await run_triage()

## Activity 4: Visualize Your Agent Architecture

Let's visualize the multi-agent system we just created.

In [ ]:
!pip install "openai-agents[viz]" -q

from agents.extensions.visualization import draw_graph

# Add a tool to one of the specialists for a more interesting graph
@function_tool
def get_tracking_status(tracking_number: str) -> str:
    ""Gets the status of a shipping package.""
    return f"Package {tracking_number} is currently in transit in a different castle."

shipping_agent_with_tool = shipping_agent.clone(tools=[get_tracking_status])

# Recreate the triage agent with the updated specialist
triage_agent_for_viz = Agent(
    name="Triage Agent",
    instructions="You are the first point of contact. Understand the user's needs and hand them off to the correct specialist.",
    handoffs=[shipping_agent_with_tool, product_agent]
)

# Generate and display the graph
graph = draw_graph(triage_agent_for_viz)
graph

# You can also save it to a file
# graph.render('triage_agent_architecture', format='png', view=False)
# print("Graph saved to triage_agent_architecture.png")